# This notebook contains the experiments on Banknote dataset with LionForests

In [1]:
import sys
cpath = !pwd
sys.path.append('/usr/src/app/algorithms/')
sys.path.append('/usr/src/app/')

In [2]:
from lionforests import LionForests
from algorithms.simpleSurrogate import GlobalSurrogateTree, LocalSurrogateTree
from algorithms.DefragTrees.defragTrees import DefragModel
from algorithms.CHIRPS.structures import data_container
import algorithms.CHIRPS.routines as rt
import algorithms.CHIRPS.structures as strcts
from algorithms.anchor.anchor_tabular import AnchorTabularExplainer

from scipy import sparse

from datasets.dataset import Dataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd 
import numpy as np
import random
np.seterr(invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import time

from sklearn.model_selection import train_test_split, LeaveOneOut, KFold
from sklearn.metrics import f1_score, precision_score

Firstly, we load the dataset and we set the feature and class names

In [3]:
banknotes = Dataset()
X, y, feature_names, class_names = banknotes.load_banknote()
#one_h_percent = int(min(10*len(X)/100,100))
#print("Instances:",one_h_percent)
new_fn = []
for i in feature_names:
    new_fn.append(i.replace(' ','_'))
feature_names = new_fn

In this set of experiments we are using this not very well written code to perform few experiments/comparisons though different datasets and algorithms!

In [4]:
def measure(X_train, X_test, y_train, y_test, feature_names, class_names, tech=False, random_state=10):
    parameters = [{
        'max_depth': [10],
        'max_features': [0.75],
        'bootstrap': [True],
        'min_samples_leaf' : [1],
        'n_estimators': [500]
    }]
    scaler = MinMaxScaler(feature_range=(-1,1))
    lf = LionForests(None, False, scaler, feature_names, class_names)
    lf.fit(X_train, y_train, params=parameters)
    
    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)
    
    def techniques(model, train, y_train, predictions, test, feature_names, class_names, lf, task, random_state=10):

        #BaselineTechnique ==============================================================================
        gt = GlobalSurrogateTree(train, predictions, feature_names, task, random_state)
        lt = LocalSurrogateTree(train, predictions, feature_names, task, 150, random_state)

        #DefragTechnique ================================================================================
        Kmax = 10
        splitter = DefragModel.parseSLtrees(model) # parse sklearn tree ensembles into the array of (feature index, threshold)
        mdl = DefragModel(modeltype=task, maxitr=100, qitr=0, tol=1e-6, restart=20, verbose=0, seed=random_state)
        mdl.fit(train, predictions, splitter, Kmax, fittype='FAB', featurename=feature_names)
        def def_cov(instances):
            ts = time.time()
            score, cover, coll = mdl.evaluate(instances[1:],lf.model.predict(instances[1:]))
            def_predictions = mdl.predict(instances[1:])
            length, nodes = mdl.find_rule_length(instances[0])
            max_len = len(length)
            comp = {1:'>',0:'<='}
            rules = {}
            for f in feature_names:
                rules[f] = []
            counter = 0
            for rule, node in list(zip(mdl.rule_,nodes)):
                for conj in range(len(node)):
                    if node[conj] and counter<=max_len-1:
                        rules[feature_names[rule[conj][0]-1]].append([comp[rule[conj][1]],rule[conj][2]])
                        counter += 1
            new_rules = {}
            for k, v in rules.items():
                if len(v) == 1:
                    new_rules[k] = v
                else:
                    mmin = None
                    mmax = None
                    for value in v:
                        if value[0]=='<=':
                            if mmin is None or mmin > value[1]:
                                mmin = value[1]
                        if value[0]=='>':
                            if mmax is None or mmax < value[1]:
                                mmax = value[1]
                    if mmin is not None:
                        new_rules[k] = [['<=', mmin]]
                    if mmax is not None:
                        new_rules[k] = [['>', mmax]]
            te = time.time()
            #print(mdl)
            return length, cover, def_predictions, te-ts, new_rules

        #Anchors =======================================================================================
        explainer = AnchorTabularExplainer(class_names, feature_names, train)
        def anchors_method(instance):
            exp = explainer.explain_instance(instance, lf.model.predict, threshold=0.95, random_state=random_state)
            anchors_dict = {}
            for i in exp.names():
                terms = i.split(' ')
                if len(terms) == 3:
                    anchors_dict[terms[0]] = [[terms[1],float(terms[2])]]
                else:
                    anchors_dict[terms[2]] = [[terms[3],float(terms[4])],[terms[1],float(terms[0])]]
            return anchors_dict, 0

        #CHIRPS =======================================================================================
        project_dir = 'C:\\Users\\iamollas\\Downloads\\LionForests Journal\\algorithms\\CHIRPS'
        temp_frame = pd.DataFrame(np.hstack((train,y_train.reshape(len(y_train),1))),columns=feature_names+['class'])
        temp_frame['class']=temp_frame['class'].astype(int)

        temp_frame = temp_frame.replace({"class": {1: 2}})
        temp_frame = temp_frame.replace({"class": {0: 1}})

        mydata = data_container(
                data = temp_frame, class_col = 'class', var_names = ['variance', 'skew', 'curtosis', 'entropy'],
                project_dir = project_dir, save_dir = 'banknote', random_state=random_state)
        meta_data = mydata.get_meta()
        f_walker = strcts.classification_trees_walker(forest=model, meta_data=meta_data)
        f_walker.forest_walk(instances = test, labels = model.predict(test), forest_walk_async = True)

        explanations = strcts.CHIRPS_container(f_walker.path_detail,
                                        forest=model,
                                        sample_instances=sparse.csr_matrix(train), # any representative sample can be used
                                        sample_labels=predictions,
                                        meta_data=meta_data)
        chts = time.time()
        explanations.run_explanations(target_classes=model.predict(test), # we're explaining the prediction, not the true label!
                                explanation_async=False,
                                random_state=random_state,
                                which_trees='majority',
                                alpha_paths=0.0,
                                support_paths=0.1,
                                score_func=1,
                                precis_threshold=0.99,
                                disc_path_bins=4,
                                merging_bootstraps=20,
                                pruning_bootstraps=20,
                                delta=0.2,
                                weighting='kldiv')
        chte = (time.time() - chts)/len(test)

        def chirps_method(idx):
            chirps_dict = {}
            for i in explanations.explainers[idx].pruned_rule:
                if i[1]:
                    chirps_dict[i[0]] = [['<=',float(i[2])]]
                else:
                    chirps_dict[i[0]] = [['>',float(i[2])]]
            return chirps_dict, 0, chte

        #LionForests
        def lf_rule(instance):
            temp = lf.explain(instance, instance_random_state=random_state)[5]
            rule = {}
            for key,value in temp.items():
                rule[key] = [['<=',value[1]],['>',value[0]]]
            return rule

        return {'gs':gt.rule,'ls':lt.rule,'an':anchors_method,'lf':lf_rule, 'df': def_cov, 'ch':chirps_method}
    interpretation = techniques(lf.model, train, y_train, predictions, test, feature_names, class_names, lf, 'classification', random_state)
    if tech:
        return interpretation, lf
    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
                    if instance[k] <= rule[feature][1][1]:#THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
        return 1
    def rule_cov_LF(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
                    if instance[k] < rule[feature][1][1]:#THIS <=
                        return 0
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        return 0
                else:
                    if instance[k] > rule[feature][0][1]: #<=
                        return 0
        return 1

    rule_generator = interpretation
    full_coverage = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    rule_length = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    time_response = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    rules = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}

    x_train_temp = train
    x_test_temp = test

    y_train_temp = predictions
    y_test_temp = test_predictions
    
    x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
    x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)   

    for tesd_ind in range(len(test)):
        for name, method in rule_generator.items():
            if name == 'ch':
                rule, op, te = method(tesd_ind)
                coverage = 0
                precision = []
                co = 0
                for i in x_test_temp:
                    res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[tesd_ind], y_test_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp)
                rules['ch'].append(rule)
            elif name == 'df':
                #FIX RULE LENGTH!
                rule, cover, predictions, te, new_rules = method(np.concatenate((x_test_temp[tesd_ind:tesd_ind+1],x_test_temp)))
                f_precision[name].append(precision_score(predictions,y_test_temp,average='micro'))
                full_coverage[name] = full_coverage[name] + cover
                rules['df'].append(new_rules)
            elif name == 'lf':
                ts = time.time()
                rule = method(x_test_temp_lf[tesd_ind])
                te = time.time() - ts
                coverage = 0
                precision = []
                co = 0
                for i in x_test_temp_lf:
                    res = rule_cov_LF(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[tesd_ind], y_test_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp_lf)
                rules['lf'].append(rule)
            else:
                ts = time.time()
                rule, prediction = method(x_test_temp[tesd_ind])
                te = time.time() - ts
                coverage = 0
                precision = []
                co = 0 
                for i in x_test_temp:
                    if name == 'anchors':
                        res = rule_cov(i, feature_names, rule)
                    else:    
                        res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1 and name=='anchors':
                        precision.append([y_test_temp[tesd_ind], y_test_temp[co]])
                    elif res == 1:
                        precision.append([prediction, y_test_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_test_temp)
                rules[name].append(rule)
            time_response[name].append(te)
            rule_length[name] = rule_length[name] + len(rule)
    return rule_generator, full_coverage, rule_length, f_precision, time_response, rules

In [5]:
total_results = []
test_size_2 = []
for rand in [7, 10, 77]:
    total_results2 = []
    kf = KFold(n_splits=10)
    folds = 0
    test_size = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        test_size.append(len(X_test))
        print('# of Fold: ' + str(folds+1) + ', size of test: ' + str(len(X_test)))    
        results = measure(X_train, X_test, y_train, y_test, feature_names, class_names, random_state=rand)
        folds=folds+1
        total_results2.append(results)
    test_size_2.append(test_size)
    total_results.append(total_results2)

# of Fold: 1, size of test: 138
len self.path_detail
138
Working on CHIRPS for instance 0 of 138
as_chirps for batch_idx 0
start mining for batch_idx 0 with support = 0.1
found 9 patterns from 500 trees for batch_idx 0
start score sort for batch_idx 0 (9) patterns
start merge rule for batch_idx 0 (9) patterns
[('variance', False, 0.04431), ('curtosis', False, -0.92493)]
0.8970840480274442 0.42502617384735675 1.0 1.0
merge complete for batch_idx 0 (9) patterns
start get explainer for batch_idx 0
as_chirps for batch_idx 1
start mining for batch_idx 1 with support = 0.1
found 9 patterns from 500 trees for batch_idx 1
start score sort for batch_idx 1 (9) patterns
start merge rule for batch_idx 1 (9) patterns
[('variance', False, 0.04431), ('curtosis', False, -0.91968)]
0.8994800693240901 0.4221609658480989 1.0 0.9999999999999999
merge complete for batch_idx 1 (9) patterns
start get explainer for batch_idx 1
as_chirps for batch_idx 2
start mining for batch_idx 2 with support = 0.1
reduced 5

[('variance', False, 0.1608)]
0.9257731958762887 0.3686974038193938 0.20745264044512884 0.3234894113075578
merge complete for batch_idx 18 (49) patterns
start get explainer for batch_idx 18
as_chirps for batch_idx 19
start mining for batch_idx 19 with support = 0.1
found 8 patterns from 500 trees for batch_idx 19
start score sort for batch_idx 19 (8) patterns
start merge rule for batch_idx 19 (8) patterns
[('skew', False, 0.43128), ('variance', False, -0.49575)]
0.9907120743034056 0.25907337954066556 0.4451332053442582 0.31535580524344575
merge complete for batch_idx 19 (8) patterns
start get explainer for batch_idx 19
Working on CHIRPS for instance 20 of 138
as_chirps for batch_idx 20
start mining for batch_idx 20 with support = 0.1
found 9 patterns from 500 trees for batch_idx 20
start score sort for batch_idx 20 (9) patterns
start merge rule for batch_idx 20 (9) patterns
[('variance', False, 0.04431), ('curtosis', False, -0.92493)]
0.8970840480274442 0.42502617384735675 1.0 1.0
merg

[('curtosis', False, -0.83592), ('variance', False, 0.04454)]
0.9080459770114943 0.38867169381237 1.0 0.9999999999999999
merge complete for batch_idx 36 (8) patterns
start get explainer for batch_idx 36
as_chirps for batch_idx 37
start mining for batch_idx 37 with support = 0.1
reduced 2 patterns out of 15 by numeric redundancy
found 13 patterns from 500 trees for batch_idx 37
start score sort for batch_idx 37 (13) patterns
start merge rule for batch_idx 37 (13) patterns
[('variance', False, 0.08179), ('curtosis', False, -0.70274)]
0.974293059125964 0.30875067919477317 0.6492367456812905 0.7542297417631345
merge complete for batch_idx 37 (13) patterns
start get explainer for batch_idx 37
as_chirps for batch_idx 38
start mining for batch_idx 38 with support = 0.1
reduced 8 patterns out of 18 by numeric redundancy
found 10 patterns from 408 trees for batch_idx 38
start score sort for batch_idx 38 (10) patterns
start merge rule for batch_idx 38 (10) patterns
[('variance', False, 0.2984), 

[('variance', False, 0.04431), ('curtosis', False, -0.91968)]
0.8994800693240901 0.4221609658480989 1.0 0.9999999999999999
merge complete for batch_idx 54 (9) patterns
start get explainer for batch_idx 54
Working on CHIRPS for instance 55 of 138
as_chirps for batch_idx 55
start mining for batch_idx 55 with support = 0.1
reduced 2 patterns out of 15 by numeric redundancy
found 13 patterns from 500 trees for batch_idx 55
start score sort for batch_idx 55 (13) patterns
start merge rule for batch_idx 55 (13) patterns
[('variance', False, 0.08179), ('curtosis', False, -0.70274)]
0.974293059125964 0.30875067919477317 0.6490404241045087 0.7536682970208981
merge complete for batch_idx 55 (13) patterns
start get explainer for batch_idx 55
as_chirps for batch_idx 56
start mining for batch_idx 56 with support = 0.1
found 8 patterns from 500 trees for batch_idx 56
start score sort for batch_idx 56 (8) patterns
start merge rule for batch_idx 56 (8) patterns
[('curtosis', False, -0.83592), ('varianc

[('variance', False, 0.04431), ('curtosis', False, -0.92505)]
0.8970840480274442 0.42502617384735675 0.9277248710938568 0.8649940262843487
merge complete for batch_idx 72 (13) patterns
start get explainer for batch_idx 72
as_chirps for batch_idx 73
start mining for batch_idx 73 with support = 0.1
reduced 5 patterns out of 18 by numeric redundancy
found 13 patterns from 500 trees for batch_idx 73
start score sort for batch_idx 73 (13) patterns
start merge rule for batch_idx 73 (13) patterns
[('variance', False, 0.13169), ('curtosis', False, -0.72614)]
0.9863013698630136 0.29200869369309673 0.7160147141621667 0.8435419440745671
merge complete for batch_idx 73 (13) patterns
start get explainer for batch_idx 73
as_chirps for batch_idx 74
start mining for batch_idx 74 with support = 0.1
found 10 patterns from 500 trees for batch_idx 74
start score sort for batch_idx 74 (10) patterns
start merge rule for batch_idx 74 (10) patterns
[('skew', False, 0.42867), ('variance', False, -0.49575)]
0.9

...

[('variance', True, -0.01852), ('skew', True, 0.60405)]
0.9381443298969072 0.302791311021502 0.47156124328526067 0.5752066115702481
merge complete for batch_idx 57 (30) patterns
start get explainer for batch_idx 57
as_chirps for batch_idx 58
start mining for batch_idx 58 with support = 0.1
reduced 4 patterns out of 26 by numeric redundancy
found 22 patterns from 500 trees for batch_idx 58
start score sort for batch_idx 58 (22) patterns
start merge rule for batch_idx 58 (22) patterns
[('variance', True, -0.03388), ('skew', True, 0.60999)]
0.9421052631578948 0.29731723760701156 0.5298596854972422 0.6351791530944626
merge complete for batch_idx 58 (22) patterns
start get explainer for batch_idx 58
as_chirps for batch_idx 59
start mining for batch_idx 59 with support = 0.1
found 16 patterns from 490 trees for batch_idx 59
start score sort for batch_idx 59 (16) patterns
start merge rule for batch_idx 59 (16) patterns
[('variance', True, -0.6318)]
0.9512195121951219 0.03150242326332795 0.285

[('variance', True, -0.04169)]
0.8243243243243243 0.32141842528165404 0.32020502726451905 0.35666549419627147
merge complete for batch_idx 74 (36) patterns
start get explainer for batch_idx 74
Working on CHIRPS for instance 75 of 137
as_chirps for batch_idx 75
start mining for batch_idx 75 with support = 0.1
reduced 5 patterns out of 35 by numeric redundancy
found 30 patterns from 500 trees for batch_idx 75
start score sort for batch_idx 75 (30) patterns
start merge rule for batch_idx 75 (30) patterns
[('skew', True, 0.46828), ('variance', True, -0.2327)]
0.9770642201834863 0.17378805032246578 0.45138868199664073 0.5754254398615517
merge complete for batch_idx 75 (30) patterns
start get explainer for batch_idx 75
as_chirps for batch_idx 76
start mining for batch_idx 76 with support = 0.1
reduced 5 patterns out of 35 by numeric redundancy
found 30 patterns from 500 trees for batch_idx 76
start score sort for batch_idx 76 (30) patterns
start merge rule for batch_idx 76 (30) patterns
[('s

[('skew', True, 0.60225), ('variance', True, -0.03363)]
0.941952506596306 0.29653068406836863 0.5089788540808786 0.6326582927454496
merge complete for batch_idx 91 (28) patterns
start get explainer for batch_idx 91
as_chirps for batch_idx 92
start mining for batch_idx 92 with support = 0.1
reduced 3 patterns out of 50 by numeric redundancy
found 47 patterns from 410 trees for batch_idx 92
start score sort for batch_idx 92 (47) patterns
start merge rule for batch_idx 92 (47) patterns
[('skew', True, 0.41074), ('variance', True, 0.06959)]
0.9136690647482014 0.32026085592289655 0.5621762858924193 0.5468347915594441
merge complete for batch_idx 92 (47) patterns
start get explainer for batch_idx 92
as_chirps for batch_idx 93
start mining for batch_idx 93 with support = 0.1
reduced 12 patterns out of 52 by numeric redundancy
found 40 patterns from 479 trees for batch_idx 93
start score sort for batch_idx 93 (40) patterns
start merge rule for batch_idx 93 (40) patterns
[('skew', True, -0.3633

[('skew', True, 0.39472), ('variance', True, -0.00586)]
0.9378378378378378 0.2890616055868622 0.6011398809980132 0.6448467966573818
merge complete for batch_idx 108 (38) patterns
start get explainer for batch_idx 108
as_chirps for batch_idx 109
start mining for batch_idx 109 with support = 0.1
found 57 patterns from 381 trees for batch_idx 109
start score sort for batch_idx 109 (57) patterns
start merge rule for batch_idx 109 (57) patterns
[('curtosis', True, -0.54543), ('skew', True, 0.42089), ('variance', True, 0.25328)]
0.9848484848484849 0.15811210190002503 0.2268765931915015 0.20912439784641554
merge complete for batch_idx 109 (57) patterns
start get explainer for batch_idx 109
Working on CHIRPS for instance 110 of 137
as_chirps for batch_idx 110
start mining for batch_idx 110 with support = 0.1
found 20 patterns from 401 trees for batch_idx 110
start score sort for batch_idx 110 (20) patterns
start merge rule for batch_idx 110 (20) patterns
[('curtosis', True, -0.92638), ('skew',

[('skew', True, 0.39443), ('variance', True, 0.05025)]
0.9269521410579346 0.30775762278503555 0.5078624106849452 0.4985133795837462
merge complete for batch_idx 125 (41) patterns
start get explainer for batch_idx 125
as_chirps for batch_idx 126
start mining for batch_idx 126 with support = 0.1
reduced 7 patterns out of 34 by numeric redundancy
found 27 patterns from 498 trees for batch_idx 126
start score sort for batch_idx 126 (27) patterns
start merge rule for batch_idx 126 (27) patterns
[('skew', True, 0.56163), ('variance', True, -0.04311)]
0.9505494505494505 0.2862673264387994 0.528252215887418 0.5882209783402287
merge complete for batch_idx 126 (27) patterns
start get explainer for batch_idx 126
as_chirps for batch_idx 127
start mining for batch_idx 127 with support = 0.1
reduced 7 patterns out of 32 by numeric redundancy
found 25 patterns from 498 trees for batch_idx 127
start score sort for batch_idx 127 (25) patterns
start merge rule for batch_idx 127 (25) patterns
[('variance

In [6]:
from utilities.lionforests_utility import path_similarity
rule_variance = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}


kf = KFold(n_splits=10)
folds = 0
test_size = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    min_max_feature_values = {}
    for i in range(len(feature_names)):
            min_max_feature_values[feature_names[i]] = [min(X_train[:, i]), max(X_train[:, i])] 
    for name in ['gs', 'ls', 'an', 'lf', 'df', 'ch']:
        for k in range(len(total_results[0][folds][-1][name])):
            r1 = total_results[0][folds][-1][name][k]
            r2 = total_results[1][folds][-1][name][k]
            r3 = total_results[2][folds][-1][name][k]
            rule_variance[name].append((path_similarity(r1, r2, feature_names, min_max_feature_values)+
                                           path_similarity(r1, r3, feature_names, min_max_feature_values)+
                                           path_similarity(r2, r3, feature_names, min_max_feature_values))/3)
    folds = folds + 1

In [7]:
f_full_coverage = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
f_rule_length = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
f_f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
f_f_time = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
for b in range(3):
    full_coverage = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
    rule_length = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
    f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    f_time = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}
    k = 0
    for i in total_results[b]:
        for name, method in i[0].items():
            full_coverage[name].append(i[1][name]/test_size_2[b][k])
            rule_length[name].append(i[2][name]/test_size_2[b][k])
            f_precision[name].append(np.array(i[3][name]).mean())
            f_time[name].append(np.array(i[4][name]).mean())
        k = + 1
    for name, method in i[0].items():
        f_full_coverage[name].append(np.array(full_coverage[name]).mean())
        f_rule_length[name].append(np.array(rule_length[name]).mean())
        f_f_precision[name].append(np.array(f_precision[name]).mean())
        f_f_time[name].append(np.array(f_time[name]).mean())
for name, method in i[0].items():
    print(name,  '| %5.4f  %5.3f | %5.4f %5.3f | %5.4f  %5.3f | %5.4f  %5.3f | %5.4f  %5.3f' 
          % (np.array(f_full_coverage[name]).mean(),np.array(f_full_coverage[name]).std(),
             np.array(f_rule_length[name]).mean(),np.array(f_rule_length[name]).std(),
             np.array(f_f_precision[name]).mean(),np.array(f_f_precision[name]).std(),
             np.array(f_f_time[name]).mean(),np.array(f_f_time[name]).std(),
             np.array(rule_variance[name]).mean(),np.array(rule_variance[name]).std()))

gs | 0.1600  0.017 | 2.9336 0.055 | 0.9955  0.003 | 0.0003  0.000 | 0.4574  0.188
ls | 0.5084  0.015 | 1.4831 0.038 | 0.9644  0.001 | 2.9819  0.019 | 0.6906  0.258
an | 0.3036  0.003 | 2.1070 0.030 | 0.5538  0.000 | 4.1935  0.152 | 0.9618  0.083
lf | 0.0651  0.002 | 2.7432 0.002 | 1.0000  0.000 | 1.0433  0.004 | 0.8326  0.150
df | 0.9862  0.001 | 5.8442 0.475 | 0.9079  0.010 | 0.2830  0.001 | 0.7153  0.162
ch | 0.5627  0.004 | 1.7312 0.019 | 0.9928  0.000 | 0.3273  0.003 | 0.9788  0.056


In [9]:
import csv  

with open('banknot.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    for name, method in i[0].items():
        writer.writerow([name, np.array(f_full_coverage[name]).mean(),np.array(f_full_coverage[name]).std(),
             np.array(f_rule_length[name]).mean(),np.array(f_rule_length[name]).std(),
             np.array(f_f_precision[name]).mean(),np.array(f_f_precision[name]).std(),
             np.array(f_f_time[name]).mean(),np.array(f_f_time[name]).std(),
             np.array(rule_variance[name]).mean(),np.array(rule_variance[name]).std()])

# Conclusiveness Check:

Let's test a few examples!

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10, stratify=y, random_state=123)
techniques = measure(X_train, X_test, y_train, y_test, feature_names, class_names, True)

We prepare our explanation techniques. They are trained in almost the whole train set

In [482]:
lf = techniques[1]
techniques = techniques[0]
X_test_scaled = lf.utilizer.transform(X_test)
X_train_scaled = lf.utilizer.transform(X_train)

We select the instance 9

In [487]:
temp_id = 9
temp = X_test[temp_id].copy()
temp_scaled = lf.utilizer.transform([X_test[temp_id],X_test[temp_id]])[0]
temp, temp_scaled

(array([ 3.583  , -3.7971 ,  3.4391 , -0.12501]),
 array([ 0.53244056, -0.25342473, -0.24826502,  0.53180938]))

Know let's generate 

In [488]:
techniques['ls'](temp_scaled) #faulty for for id 9, temp2[2] = -5 for id 8, id 5, curt id 4

({}, 0)

Local Surrogate said 0 without giving any info

In [489]:
techniques['an'](temp_scaled) #faulty for temp2[2] = -5 for id 9

({'variance': [['>', 0.42]]}, 0)

Anchors said if variance > 0.42

In [490]:
techniques['ch'](temp_id) #faulty for temp2[2] = -5 for id 9 and id 8 amd od 4, temp2[1] = -13.7731 for id 5

({'variance': [['>', 0.23071]]}, 0)

CHIRPS Said 0 variance > 0.23071

In [491]:
techniques['gs'](temp_scaled)

({'skew': [['<=', 0.66949972230517]],
  'variance': [['>', 0.1280646916062096]],
  'curtosis': [['>', -0.6443884608512067]]},
 0)

The Global Suroggate said 0 because variance > 0.1280646916062096 and skew <= 0.66949972230517 and curtosis > -0.6443884608512067

In [492]:
techniques['lf'](temp)

{'variance': [['<=', 6.8248], ['>', 2.3942999407537267]],
 'curtosis': [['<=', 17.9274], ['>', -0.8297101872697472]]}

Finally, LF said 0 if 0.36099632 < variance < 1 and -0.61605188 < curtosis < 1. Those values are transformed, because LF provides ranges for the real values of an instance and not for the scaled values.

In [498]:
temp2 = temp.copy()
temp2[2] = -5 #Let's change the curtosis value and put it to the min value. Scaled value is -1 
temp2_scaled = lf.utilizer.transform([temp2,temp2])[0]
lf.model.predict([temp_scaled,temp2_scaled])

array([0, 1], dtype=int64)

Local surrogate did not gave any valuable info as well. Anchors and CHIRPS did not provided any condition for the curtosis feature. And when we modified it's value the prediction changed. Therefore, we can say that the two rules were inconclusive. Global Surrogate provided a conclusive answer, like LF. 

In [500]:
temp_id = 6
temp = X_test[temp_id].copy()
temp_scaled = lf.utilizer.transform([X_test[temp_id],X_test[temp_id]])[0]
temp, temp_scaled

(array([ 3.5156, 10.1891, -4.2759, -4.978 ]),
 array([ 0.52271957,  0.79326241, -0.91296444, -0.35073697]))

In [485]:
techniques['df'](np.array([temp_scaled,temp_scaled, temp_scaled, temp_scaled]))

[Rule  1]
y = 0 when
	 skew > 0.337126
	 curtosis > -0.974536

[Rule  2]
y = 0 when
	 variance > 0.078377
	 skew <= 0.400540
	 curtosis > -0.726306
	 entropy > 0.160262

[Rule  3]
y = 1 when
	 variance <= 0.118145
	 skew <= 0.301429
	 curtosis > -0.484685

[Rule  4]
y = 1 when
	 -0.645371 < variance <= 0.308764
	 -0.135811 < skew <= 0.420517
	 curtosis <= -0.274181
	 entropy <= 0.777904

[Otherwise]
y = 0



([0, 0], 1.0, array([0, 0, 0], dtype=int64))

In [484]:
techniques['lf'](temp)

{'variance': [['<=', 6.8248], ['>', 2.8434999517112955]],
 'curtosis': [['<=', -4.02399996997267], ['>', -4.364150089024007]]}

In [505]:
temp2 = temp.copy()
temp2[0] = -7 #Let's change the curtosis value and put it to the min value. Scaled value is -1 
temp2_scaled = lf.utilizer.transform([temp2,temp2])[0]
lf.model.predict([temp_scaled,temp2_scaled])

array([0, 1], dtype=int64)

Let's try one more example to prove that GS is also inconclusive

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=10, stratify=y, random_state=77)
techniques = measure(X_train, X_test, y_train, y_test, feature_names, class_names, True)

In [465]:
lf = techniques[1]
techniques = techniques[0]
X_test_scaled = lf.utilizer.transform(X_test)
X_train_scaled = lf.utilizer.transform(X_train)

We select a random instance again

In [466]:
temp_id = 0
temp = X_test[temp_id].copy()
temp_scaled = lf.utilizer.transform([X_test[temp_id],X_test[temp_id]])[0]
temp, temp_scaled

(array([-1.6936,  2.7852, -2.1835, -1.9276]),
 array([-0.22859471,  0.23917574, -0.73269003,  0.2039972 ]))

In [467]:
techniques['gs'](temp_scaled)

({'variance': [['<=', 0.05603344863243409], ['>', -0.25483223672374644]],
  'skew': [['<=', 0.32315588433361764], ['>', 0.22769872546014208]]},
 1)

GS said 1 if -0.2548322 < Variance <= 0.05603344 and 0.2276987254 < skew <= 0.32315588433361764

In [468]:
techniques['lf'](temp)

{'variance': [['<=', -0.966239995934629], ['>', -1.698349986315147]],
 'skew': [['<=', 3.9254000015519557], ['>', -13.773100000000001]],
 'curtosis': [['<=', -0.2843348647490144], ['>', -5.286099999999999]]}

And LF said 0 because: -0.229 < variance <= -0.12368878 and -1 < skew <= 0.32 and -1 < curtosis <= -0.56906411

In [480]:
temp2 = temp.copy()

temp2[2] = 17.9274 

temp2_scaled = lf.utilizer.transform([temp2,temp2])[0]
lf.model.predict([temp_scaled,temp2_scaled])

array([1, 0], dtype=int64)

We changed the value of curtosis, which was not iuncluded in GS's rule and the prediction changed. GS's rule therefore it was not conclusive